In [1]:
%load_ext cudf.pandas
import pandas as pd
import importlib
import workbench.data_loader as data_loader
import workbench.data_process as data_process
import numpy as np
import data_process

UsageError: Line magic function `%load_ext("cudf.pandas")` not found.


In [5]:
pd.set_option("max_colwidth", 8000)
pd.options.display.max_rows = 1000
pd.set_option("display.width", 500)
pd.set_option("display.max_colwidth", 5000)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", -1)

pd.set_option("display.max_columns", 200)

/tmp/ipykernel_9480/2955908732.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [71]:
importlib.reload(data_loader)
data = data_loader.load_data()

In [7]:
importlib.reload(data_process)
dual_df = data_process.append_rolling_match_team_stats(data.matches_df_short)

In [8]:
dual_df

,id,days_after_first_date,match_api_id,result,goals_scored,goals_conceded,team_id,opponent_id,is_home_team,goal_deficit,rolling_mean_goals_scored,rolling_mean_goals_conceded,rolling_mean_goal_deficit
4768,4769,22,483129,1,2,1,8583,9830,1,1,NaN,NaN,NaN
30747,4769,22,483129,-1,1,2,9830,8583,0,-1,NaN,NaN,NaN
30748,4770,22,483130,-1,1,2,7819,9827,0,-1,NaN,NaN,NaN
4769,4770,22,483130,1,2,1,9827,7819,1,1,NaN,NaN,NaN
4770,4771,22,483131,1,1,0,9746,9831,1,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47457,21479,2858,2216670,1,4,0,8649,8485,0,4,0.8,1.2,-0.4
21479,21480,2858,2216671,1,7,0,9925,9927,1,7,2.0,1.4,0.6
47458,21480,2858,2216671,-1,0,7,9927,9925,0,-7,1.4,1.8,-0.4
47459,21481,2858,2216672,0,2,2,8467,9860,0,0,1.6,1.0,0.6


In [9]:
import pandas as pd


def calculate_elo_ratings(data):
    K = 30
    initial_rating = 1500

    # Initialize ratings dictionary
    ratings = {}

    for _, row in data.iterrows():
        team_id = row["team_id"]
        opponent_id = row["opponent_id"]
        result = row["result"]

        # Initialize teams' ratings if not already present
        if team_id not in ratings:
            ratings[team_id] = initial_rating
        if opponent_id not in ratings:
            ratings[opponent_id] = initial_rating

        # Calculate expected scores
        E_team = 1 / (1 + 10 ** ((ratings[opponent_id] - ratings[team_id]) / 400))
        E_opponent = 1 / (1 + 10 ** ((ratings[team_id] - ratings[opponent_id]) / 400))

        # Actual scores
        S_team = 0.5 + 0.5 * result
        S_opponent = 0.5 - 0.5 * result

        # Update ratings
        ratings[team_id] += K * (S_team - E_team)
        ratings[opponent_id] += K * (S_opponent - E_opponent)

    return ratings

In [10]:
elo_ratings = calculate_elo_ratings(dual_df)

In [37]:
elo_ratings_df = pd.DataFrame.from_dict(elo_ratings, orient="index").reset_index()
elo_ratings_df = elo_ratings_df.rename(columns={"index": "team_api_id", 0: "Elo"})
elo_ratings_df

,team_api_id,Elo
0,8583.0,1421.720180
1,9830.0,1590.069068
2,7819.0,1576.068446
3,9827.0,1555.195142
4,9746.0,1323.685848
5,9831.0,1602.942365
6,8682.0,1407.020919
7,8689.0,1530.098757
8,9748.0,1730.222626
9,9941.0,1498.178052


In [50]:
data.team_attrs_df

,id,team_fifa_api_id,team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,days_after_first_date,team_long_name,league_name
963,1113,874,1601,30,Slow,NaN,Little,40,Mixed,Organised,50,Normal,35,Normal,70,Lots,Organised,65,Medium,60,Press,50,Normal,Cover,0,Ruch Chorzów,Switzerland Super League
967,1114,874,1601,48,Balanced,NaN,Little,51,Mixed,Organised,68,Risky,67,Lots,51,Normal,Organised,46,Medium,48,Press,50,Normal,Cover,365,Ruch Chorzów,Switzerland Super League
971,1115,874,1601,53,Balanced,NaN,Little,55,Mixed,Organised,44,Normal,65,Normal,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover,730,Ruch Chorzów,Switzerland Super League
975,1116,874,1601,53,Balanced,NaN,Little,55,Mixed,Organised,44,Normal,65,Normal,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover,1306,Ruch Chorzów,Switzerland Super League
965,1117,874,1601,53,Balanced,48.0,Normal,38,Mixed,Organised,66,Normal,65,Normal,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover,1670,Ruch Chorzów,Switzerland Super League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076,69,112513,158085,69,Fast,66.0,Normal,39,Mixed,Organised,55,Normal,59,Normal,46,Normal,Organised,35,Medium,37,Press,37,Normal,Cover,1670,FC Arouca,Switzerland Super League
1080,70,112513,158085,65,Balanced,66.0,Normal,39,Mixed,Organised,55,Normal,59,Normal,46,Normal,Organised,37,Medium,39,Press,37,Normal,Cover,2026,FC Arouca,Switzerland Super League
746,275,112409,208931,32,Slow,46.0,Normal,31,Short,Organised,47,Normal,36,Normal,54,Normal,Organised,46,Medium,44,Press,51,Normal,Cover,1670,Carpi,Switzerland Super League
750,276,112409,208931,80,Fast,45.0,Normal,65,Mixed,Organised,70,Risky,40,Normal,50,Normal,Organised,25,Deep,55,Press,35,Normal,Cover,2026,Carpi,Switzerland Super League


In [73]:
team_info_df = data.team_attrs_df[
    ["team_api_id", "team_fifa_api_id", "team_long_name", "league_name"]
]

In [52]:
# elo_ratings_df["team_api_id"]
elo_ratings_df

,team_api_id,Elo
0,8583.0,1421.720180
1,9830.0,1590.069068
2,7819.0,1576.068446
3,9827.0,1555.195142
4,9746.0,1323.685848
5,9831.0,1602.942365
6,8682.0,1407.020919
7,8689.0,1530.098757
8,9748.0,1730.222626
9,9941.0,1498.178052


In [74]:
team_elo_df = elo_ratings_df.merge(team_info_df, on="team_api_id")
team_elo_df = team_elo_df.sort_values(by="Elo", ascending=False).drop_duplicates()
team_elo_df

,team_api_id,Elo,team_fifa_api_id,team_long_name,league_name
468,9772.0,1938.097918,234,SL Benfica,Portugal Liga ZON Sagres
309,9768.0,1902.571640,237,Sporting CP,Portugal Liga ZON Sagres
1093,8633.0,1885.376796,243,Real Madrid CF,Spain LIGA BBVA
1042,9823.0,1872.156014,21,FC Bayern Munich,Germany 1. Bundesliga
1308,8640.0,1870.965176,247,PSV,Netherlands Eredivisie
547,9847.0,1856.495841,73,Paris Saint-Germain,France Ligue 1
1203,9885.0,1848.195655,45,Juventus,Italy Serie A
516,8593.0,1819.098571,245,Ajax,Netherlands Eredivisie
354,8634.0,1810.454917,241,FC Barcelona,Spain LIGA BBVA
63,9906.0,1795.269517,240,Atlético Madrid,Spain LIGA BBVA


In [79]:
team_elo_df[team_elo_df["league_name"] == "England Premier League"]

,team_api_id,Elo,team_fifa_api_id,team_long_name,league_name
274,8197.0,1780.724615,95,Leicester City,England Premier League
642,8586.0,1696.620082,18,Tottenham Hotspur,England Premier League
386,8466.0,1694.950466,17,Southampton,England Premier League
1044,9825.0,1676.179310,1,Arsenal,England Premier League
899,8654.0,1665.095752,19,West Ham United,England Premier League
1452,10260.0,1646.646506,11,Manchester United,England Premier League
771,8456.0,1645.882783,10,Manchester City,England Premier League
898,8650.0,1638.146306,9,Liverpool,England Premier League
131,10003.0,1606.501934,1960,Swansea City,England Premier League
770,8455.0,1602.381733,5,Chelsea,England Premier League
